In [4]:
# Bibliotecas 

import pandas as pd 
#import joblib  # usado para deploy
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import TfidfVectorizer  
from sklearn.preprocessing import LabelEncoder

from sklearn.svm import LinearSVC #classificador 
from sklearn.pipeline import Pipeline  
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report   
from sklearn import tree 
from sklearn.naive_bayes import MultinomialNB

from google.colab import drive 
drive.mount('/gdrive')


Mounted at /gdrive


In [5]:
#from google.colab import drive
#drive.mount('/content/drive')

In [6]:
dataset = pd.read_csv("/gdrive/MyDrive/TCC2/data2.csv") # Carregando arquivo
dataset.shape # Dimensão - linhas,colunas

(30000, 5)

In [7]:
dataset.head()

,market_id,Marca,title,Marca Lett,Verificado
0,1,Acqua Panna,Água Mineral Acqua Panna,Acqua Panna,1 SIM
1,1,Acqua Panna,Água Mineral sem Gás Acqua Panna,Acqua Panna,1 SIM
2,1,Adria,Biscoito ADRIA Cream Cracker com gergelim 215g,Adria,1 SIM
3,1,Adria Plus Life,Biscoito ADRIA Plus Life Integral 156g,Adria Plus Life,1 SIM
4,1,Adria Plus Life,Biscoito Adria Plus Life Integral Aveia e Mel ...,Adria Plus Life,1 SIM


### Balanceamento

In [8]:
# Fazendo contagem de frequência das marcas e criando coluna  
dataset['frequencia'] = dataset.groupby('Marca')['Marca'].transform('count') 
dataset = dataset.sort_values(by=['frequencia'], ascending=False)  # Ordem decrescente 
dataset.head(10)

,market_id,Marca,title,Marca Lett,Verificado,frequencia
8399,1020,Colgate,COLGATE Cepillo Dental 360 Luminous White Medi...,Colgate,1020,1827
13488,105,Colgate,ESCOVA DENTAL COLGATE GENGIVA COMFORT 2 UNIDADES,Colgate,OK 105,1827
13486,105,Colgate,Escova Dental Colgate Extra Clean 1unid,Colgate,OK 105,1827
13485,105,Colgate,Escova Dental Colgate Dr. Dentuço 1unid,Colgate,OK 105,1827
13484,105,Colgate,Escova Dental Colgate Classic Clean 3 Unidades,Colgate,OK 105,1827
13483,105,Colgate,Escova Dental Colgate Bamboo,Colgate,OK 105,1827
13482,105,Colgate,Escova Dental Colgate Agnes e Fluffy 2 a 5 Ano...,Colgate,OK 105,1827
13481,105,Colgate,Escova Dental Colgate Agnes e Fluffy 6+Anos 2...,Colgate,OK 105,1827
13480,105,Colgate,Escova Dental Colgate 360º Total 12 2unid Prom...,Colgate,OK 105,1827
13479,105,Colgate,Escova Dental Colgate 360º Original 2unid Prom...,Colgate,OK 105,1827


In [9]:
# Selecionando colunas e analisando frequência de marcas
freq_marcas = dataset[['frequencia',"Marca"]]  
freq_marcas = freq_marcas.drop_duplicates() #Excluindo duplicadas para facilitar na análise
freq_marcas.head(60)

,frequencia,Marca
8399,1827,Colgate
23654,1494,Brastemp
4706,1355,Pantene
18649,1195,Gillette
11924,1182,Oral B
28304,1166,Consul
2029,1102,Pampers
5705,966,Barbie
7391,858,Palmolive
12787,766,Protex


In [10]:
#Selecionando marcas
df = dataset[dataset['frequencia'] >= 110] 

In [11]:
vetor_marcas = df['Marca'].unique()
# Pegando 100 amostras de cada marca 
# Rodar uma única vez 
final = pd.DataFrame()
for i in range(len(vetor_marcas)):
    df_mask = dataset['Marca'] == vetor_marcas[i] 
    filtered_df = dataset[df_mask].sample(100) 
    final = final.append(filtered_df)  


final

,market_id,Marca,title,Marca Lett,Verificado,frequencia
18201,114,Colgate,Creme Dental Colgate 180g Tripla Acao Original,Colgate,OK 114,1827
618,1,Colgate,Escova Dental COLGATE Colors Limpeza Efetiva c...,Colgate,1 SIM,1827
13529,105,Colgate,Gel Dental Colgate Zero Kids Sem Flúor Mix Fru...,Colgate,OK 105,1827
21797,1153,Colgate,Creme Dental Colgate Tripla Ação Xtra White 70...,Colgate,1153,1827
13382,105,Colgate,Creme Dental Colgate Luminous White Carvão Ati...,Colgate,OK 105,1827
...,...,...,...,...,...,...
13156,104,Vick,Pomada Calmante para Bebês Vick BabyRub 12g,Vick,OK 104,110
14924,105,Vick,Vick Drops Sabor Limão 5 pastilhas,Vick,OK 105,110
14913,105,Vick,Chá para Gripe Vick Pyrena Sabor Camomila 500mg,Vick,OK 105,110
8352,102,Vick,Descongestionante Vick VapoRub 12g,Vick,OK 102,110


In [12]:
# Selecionando colunas
final_2 = final[['title','Marca']] 
final_2.head()

,title,Marca
18201,Creme Dental Colgate 180g Tripla Acao Original,Colgate
618,Escova Dental COLGATE Colors Limpeza Efetiva c...,Colgate
13529,Gel Dental Colgate Zero Kids Sem Flúor Mix Fru...,Colgate
21797,Creme Dental Colgate Tripla Ação Xtra White 70...,Colgate
13382,Creme Dental Colgate Luminous White Carvão Ati...,Colgate


In [13]:
# Transformando os rótulos(Marcas) em números
le = LabelEncoder() 
Y = le.fit_transform(final_2['Marca'])


In [14]:
# Separando em treino e teste 

X_train, X_test, y_train,y_test = train_test_split (final_2['title'], Y, test_size = 0.4, random_state = 3)

In [15]:
# Linear SVC
pipeline = Pipeline ([ 
                        ('vectorizer', TfidfVectorizer()), 
                        ('clf', LinearSVC())


]) 
# Árvore de decisão
pipeline_tree = Pipeline ([ 
                        ('vectorizer', TfidfVectorizer()), 
                        ('decision', tree.DecisionTreeClassifier())


])  

# MultinomialNB  
pipeline_multi = Pipeline ([ 
                        ('vectorizer', TfidfVectorizer()), 
                        ('decision',MultinomialNB ())


]) 


In [16]:
pipeline.fit(X_train, y_train) 
pipeline_tree.fit (X_train, y_train)   
pipeline_multi.fit (X_train, y_train)  


Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('decision', MultinomialNB())])

In [17]:
pred = pipeline.predict(X_test) 
pred_tree = pipeline_tree.predict(X_test) 
pred_multi = pipeline.predict (X_test)

In [18]:
# Acurácia geral 

print("Acurácia Linear SVC:", accuracy_score(y_test, pred)) 
print("Acurácia árvore de decisão:", accuracy_score(y_test, pred_tree)) 
print("Acurácia MultinomialNB:", accuracy_score(y_test, pred_multi))

Acurácia Linear SVC: 0.9858333333333333
Acurácia árvore de decisão: 0.98125
Acurácia MultinomialNB: 0.9858333333333333


In [19]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       1.00      1.00      1.00        42
           2       1.00      1.00      1.00        39
           3       1.00      1.00      1.00        42
           4       0.97      1.00      0.99        36
           5       0.97      0.95      0.96        40
           6       0.95      1.00      0.97        39
           7       0.95      0.95      0.95        44
           8       0.97      0.97      0.97        37
           9       0.95      1.00      0.98        40
          10       0.98      1.00      0.99        48
          11       1.00      1.00      1.00        40
          12       1.00      1.00      1.00        38
          13       0.98      1.00      0.99        40
          14       0.90      1.00      0.95        37
          15       0.97      1.00      0.98        57
          16       1.00      1.00      1.00        48
          17       1.00    

In [22]:
from sklearn.model_selection import cross_val_score


scores = cross_val_score(pipeline, X_train, y_train, cv=5)
print("Scores - Linear SVC",scores)  

scores = cross_val_score(pipeline_tree, X_train, y_train, cv=5)
print("Scores - Decision Tree",scores)   

scores = cross_val_score(pipeline_multi, X_train, y_train, cv=5)
print("Scores - MultinomialNB",scores)  

Scores - Linear SVC [0.98888889 0.99166667 0.98888889 0.98194444 0.98888889]
Scores - Decision Tree [0.9875     0.98472222 0.98194444 0.98472222 0.98611111]
Scores - MultinomialNB [0.96805556 0.975      0.97361111 0.96388889 0.96388889]


In [ ]:
import matplotlib.pyplot as plt 




In [24]:

text = ["Chiclete trident sabor menta", "Limpador multiuso para banheiro veja", "Absorvente noturno always", 
        "creme dental colgate sabor menta", "chocolate garoto ao leite", "limpeza profunda multiuso lysoform", 
        "laminas de barbear gillete","fraldas noturna com maior pode de absorção pampers", "diamante negro lacta" , 
        "durma melhor com vick", "o melhor lanche para seu filho, papa mucilon"]  
marca = ["Trident", "Veja","Always","Colgate", "Garoto","Lysoform","Gillete","Pampers","Lacta","Vick","Mucilon" ]

# Testando predições
dataset = pd.DataFrame() 
linear = [] 
arvore = [] 
multi = [] 

for i in range(len(text)):
  predction = pipeline.predict([text[i]])  
  linear.append(le.inverse_transform(predction))  
  #print("Predição do Linear SVC:", le.inverse_transform(predction)) 

  predction_ = pipeline_tree.predict([text[i]])  
  arvore.append(le.inverse_transform(predction_))
  #print("Predição da árvore de decisão:", le.inverse_transform(predction_))  

  predction_m = pipeline_multi.predict([text[i]])  
  multi.append(le.inverse_transform(predction_m))
  #print("Predição MultinomialNB:", le.inverse_transform(predction_m))  
 # print("------------------------------------------------------------------")


dataset["Descrição"] = text
dataset["Linear SVC"] = linear 
dataset["Árvore de decisão"] = arvore 
dataset["Multinomial NB"] = multi   
dataset["Marca"] = marca

dataset.head(11)

,Descrição,Linear SVC,Árvore de decisão,Multinomial NB,Marca
0,Chiclete trident sabor menta,[Trident],[Trident],[Trident],Trident
1,Limpador multiuso para banheiro veja,[Veja],[Veja],[Veja],Veja
2,Absorvente noturno always,[Always],[Always],[Always],Always
3,creme dental colgate sabor menta,[Colgate],[Colgate],[Colgate],Colgate
4,chocolate garoto ao leite,[Garoto],[Garoto],[Garoto],Garoto
5,limpeza profunda multiuso lysoform,[Lysoform],[Lysoform],[Ajax],Lysoform
6,laminas de barbear gillete,[Gillette],[Halls],[Gillette],Gillete
7,fraldas noturna com maior pode de absorção pam...,[Pampers],[Pampers],[Pampers],Pampers
8,diamante negro lacta,[Lacta],[Lacta],[Lacta],Lacta
9,durma melhor com vick,[Vick],[Vick],[Vick],Vick
